# Lista 3
### NCG-040 - Introdução à Visão Computacional

**Instruções:**

 1. Comentários nos códigos desenvolvidos.
 1. Caso exista uma pergunta, deve ser respondida como comentários no arquivo ou em uma nova célula de texto.
 1. Digitar seu nome e DRE abaixo.
 1. Ao finalizarem a lista, renomeie ou salve uma cópia do arquivo no formato NomeSobrenome_ListaX.ipynb
 1. Enviar via Dropbox até 25/06/2025 - https://www.dropbox.com/request/Duz3Wq2nsdIYQVp7PVvH

`Nome: Diego Vasconcelos Schardosim de Matos`

`DRE: 120098723`

### 0) Instruções

Para esta lista, vamos fazer algo um pouco diferente. Vocês estão livres para escolher um dataset simples e fazer um treinamento de rede neural, conforme visto nas aulas do dia 22 e 29 de maio. No mínimo uma tarefa de classificação binária.

Dica: Cats vs Dogs do Kaggle ou similar